# FlatTree v.3

Вариант с классами DictKey, ListKey, соответствующими DictBranch и ListBranch, с функциями, собранными в private.py

## Термины
xtree
: eXtended Tree, hierarchical structure with dict or list branches

ixtree
: internal (or intermediate) xtree, with flattree classes as branches, which can be rendered dicts or lists

leafpath
: path to the leaf across xtree encoded as a list of Key classes, e.g. [DictKey('server'), ListKey(3)]

leafkey
: path to the leaf across xtree encoded as a string, e.g. "server[3].cpu[0].model"

leafstream
: iterable of (leafpath, leaf value) tuples

crown
: dictionary of tree leaves {leafkey: value}

shadow
: (leafkey, value) pairs that can't make into a tree because they're shadowed by the crown

### Examples of xtrees

In [1]:
xt0 = None  # degenerate "root=leaf" case
xt1 = [{'x': 'X', 
        1: 1000, 
        None: [None, {1: {'1.1': 11, '1.2': {'1.2.[1]': 121, '1.2.[2]': 122}}}]},
       {'True': True, 'False': False},
      'tail']  # root=list
xt2 = {
    '': {'': {'': xt1}},
    1: 'One',
    '1': 1,
    ".''..[]": 'Separators, brackets, quotes'
}  # root = dictionary

## Ключевой код

In [2]:
from flattree import get_default_settings
settings = get_default_settings()
settings

{'separator': '.', 'brackets': '[]', 'quotes': "'"}

### Конвертируем xtree в crown

In [3]:
from flattree.private import build_crown

In [4]:
from pprint import pprint
for xtree in (xt0, xt1, xt2):
    pprint(build_crown(xtree, settings))

{None: None}
{'[0].1': 1000,
 '[0].None[0]': None,
 "[0].None[1].1.'1.1'": 11,
 "[0].None[1].1.'1.2'.'1.2.[1]'": 121,
 "[0].None[1].1.'1.2'.'1.2.[2]'": 122,
 '[0].x': 'X',
 "[1].'False'": False,
 "[1].'True'": True,
 '[2]': 'tail'}
{"'.''''..[]'": 'Separators, brackets, quotes',
 "'1'": 1,
 '..[0].1': 1000,
 '..[0].None[0]': None,
 "..[0].None[1].1.'1.1'": 11,
 "..[0].None[1].1.'1.2'.'1.2.[1]'": 121,
 "..[0].None[1].1.'1.2'.'1.2.[2]'": 122,
 '..[0].x': 'X',
 "..[1].'False'": False,
 "..[1].'True'": True,
 '..[2]': 'tail',
 '1': 'One'}


### Конвертируем обратно: crown в xtree

Процедура восстановления xtree состоит из двух шагов:
    
1. Из Crown получаем leafstream
2. Из leafstream получаем восстановленный xtree (и пустой shadow)

Два шага необходимы потому, что шаг 2 может использоваться отдельно для восстановления xtree и получения shadow из цепочки leafstream, например, при работе операции merge

In [5]:
from flattree.private import _gen_leafstream, _from_leafstream

In [6]:
for xtree in (xt0, xt1, xt2):
    crown = build_crown(xtree, settings)
    stream = _gen_leafstream(crown, settings)
    _xtree, shadow = _from_leafstream(stream)
    print(xtree)
    print(_xtree)
    print(xtree == _xtree)

None
None
True
[{'x': 'X', 1: 1000, None: [None, {1: {'1.1': 11, '1.2': {'1.2.[1]': 121, '1.2.[2]': 122}}}]}, {'True': True, 'False': False}, 'tail']
[{'x': 'X', 1: 1000, None: [None, {1: {'1.1': 11, '1.2': {'1.2.[1]': 121, '1.2.[2]': 122}}}]}, {'True': True, 'False': False}, 'tail']
True
{'': {'': {'': [{'x': 'X', 1: 1000, None: [None, {1: {'1.1': 11, '1.2': {'1.2.[1]': 121, '1.2.[2]': 122}}}]}, {'True': True, 'False': False}, 'tail']}}, 1: 'One', '1': 1, ".''..[]": 'Separators, brackets, quotes'}
{'': {'': {'': [{'x': 'X', 1: 1000, None: [None, {1: {'1.1': 11, '1.2': {'1.2.[1]': 121, '1.2.[2]': 122}}}]}, {'True': True, 'False': False}, 'tail']}}, 1: 'One', '1': 1, ".''..[]": 'Separators, brackets, quotes'}
True


## Целевая реализация